<a href="https://colab.research.google.com/github/manhhad32/nlp/blob/master/exPhoBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Chuẩn Bị Dữ Liệu
Giả sử chúng ta có một tập dữ liệu phân loại văn bản với các nhãn "positive" và "negative".

In [2]:
# Mẫu dữ liệu
texts = ["Tôi rất thích sản phẩm này!", "Sản phẩm này thật tệ."]
labels = [1, 0]  # 1: positive, 0: negative


Tokenize Dữ Liệu

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Tạo DataLoader

In [4]:
import torch
from torch.utils.data import DataLoader, TensorDataset

labels = torch.tensor(labels)
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'], labels)
dataloader = DataLoader(dataset, batch_size=2)


Xây Dựng Mô Hình

In [5]:
import torch.nn as nn
from transformers import AutoModel

class PhoBERTClassifier(nn.Module):
    def __init__(self):
        super(PhoBERTClassifier, self).__init__()
        self.phobert = AutoModel.from_pretrained("vinai/phobert-base")
        self.classifier = nn.Linear(768, 2)  # PhoBERT base có đầu ra 768 chiều

    def forward(self, input_ids, attention_mask):
        outputs = self.phobert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :]  # Lấy embedding của token [CLS]
        logits = self.classifier(cls_output)
        return logits

model = PhoBERTClassifier()


Huấn Luyện Mô Hình

In [6]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=1e-5)

# Vòng lặp huấn luyện đơn giản
for epoch in range(3):
    for batch in dataloader:
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        loss.backward()
        optimizer.step()
        print(f"Loss: {loss.item()}")


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Loss: 0.6325210332870483
Loss: 0.4885522127151489
Loss: 0.40212178230285645
